**Predictive Modeling - 3. Time Series Forecasting**

Time series forecasting helps predict future values using historical time series data. This guide demonstrates using an Exponential Smoothing model from `statsmodels`, with proper handling of the `.predict()` method and explicit index frequency settings.

Based on [`piplist.txt`](./README.md) output, you should have `pandas`, `numpy`, `statsmodels`, `seaborn`, and `matplotlib` libraries.

**Step 1: Generate Sample Time Series Data for Forecasting**

We'll generate a longer daily time series dataset than before, with clear trend and seasonality, to make forecasting meaningful. We'll also include missing values.

In a new Excel cell, enter `=PY` and paste the following code, then press **Ctrl+Enter**:

In [ ]:
# Generate dummy daily time series data for forecasting
import pandas as pd
import numpy as np
from datetime import date, timedelta
import random

# Define date range (daily data for several years)
start_date = date(2020, 1, 1)
end_date = date(2024, 12, 31) # Forecast period included in end date for generating data
dates = pd.date_range(start=start_date, end=end_date, freq='D')

n_points = len(dates)

# Simulate components:
# 1. Trend: Linear trend with a slight curve
trend = np.linspace(100, 500, n_points) + np.sin(np.linspace(0, np.pi, n_points)) * 50

# 2. Seasonality: Weekly pattern (e.g., higher values on weekends)
day_of_week = dates.dayofweek.values
weekly_seasonality = np.sin(day_of_week * (2 * np.pi / 7)) * 30
weekly_seasonality[day_of_week >= 5] += 25 # Boost weekends

# 3. Longer-term Seasonality (e.g., yearly pattern, simplified)
yearly_seasonality = np.sin(dates.dayofyear * (2 * np.pi / 365.25)) * 40

# 4. Noise: Random fluctuations
noise = np.random.normal(0, 20, n_points) # Mean 0, Std Dev 20

# Combine components (additive model)
value = trend + weekly_seasonality + yearly_seasonality + noise

# Ensure values are non-negative
value = np.maximum(20, value) # Floor at 20

# Create DataFrame
df_ts_forecast = pd.DataFrame({'Date': dates, 'Value': value.round(2)})

# Add some missing values randomly
missing_indices = random.sample(range(n_points), int(n_points * 0.05)) # 5% missing
df_ts_forecast.loc[missing_indices, 'Value'] = np.nan


df_ts_forecast # Output the DataFrame

**Explanation:**

*   This code generates a DataFrame `df_ts_forecast` with daily data, incorporating trend, weekly, and a simplified yearly seasonality, plus noise.
*   The date range extends into 2024 to provide a period for forecasting.
*   Missing values (`np.nan`) are added to the `Value` column.
*   The result, `df_ts_forecast`, will be spilled into your Excel sheet. Let's assume this data is placed in a range or Table named `ForecastData`.

**Step 2: Prepare Data, Build, Forecast, and Evaluate Time Series Model**

Now, we'll load this dummy data, set the index frequency explicitly, handle missing values, split the data chronologically, train an `ExponentialSmoothing` model, use `.predict()` for forecasting, evaluate the forecasts, and visualize the results.

In a **new** Excel cell, enter `=PY` and paste the following code. Replace `"ForecastData"` with the actual name of the Excel range/Table where your dummy data is. Press **Ctrl+Enter**.

In [ ]:
# Prepare data, build, forecast, and evaluate Time Series model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.holtwinters import ExponentialSmoothing # For forecasting
from sklearn.metrics import mean_absolute_error, mean_squared_error # For evaluation
import warnings
from datetime import date, timedelta

# Suppress specific statsmodels warnings that can occur with fitting
warnings.filterwarnings("ignore", message="OptimizationWarning:")


# Load the data from Excel
# IMPORTANT: Replace "ForecastData" with the actual name of your Excel range or Table
df = xl("ForecastData[#All]", headers=True)

# Ensure 'Date' is a datetime column and set it as the index
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

# Ensure the index is sorted (important for time series)
df = df.sort_index()

# --- Data Preparation ---

# Explicitly set the frequency of the index as requested
# This is crucial for statsmodels models that expect a regular frequency
# 'D' stands for daily frequency
# If your original data source might skip dates, asfreq('D') will add them with NaNs
df = df.asfreq('D')
print(f"DataFrame index frequency set to: {df.index.freq}") # Print freq for confirmation

# Handle missing values in the time series data BEFORE fitting the model
# Interpolation is a common method for time series NaNs
df['Value'] = df['Value'].interpolate(method='time') # Use time-based interpolation

# Check if there are still NaNs (e.g., at the very beginning or end if interpolate can't fill)
if df['Value'].isnull().sum() > 0:
     print(f"Warning: {df['Value'].isnull().sum()} NaNs remaining after interpolation. Filling remaining with median.")
     df['Value'] = df['Value'].fillna(df['Value'].median()) # Fallback fill


# Define the split point for training and testing (e.g., last 6 months for test)
forecast_period_days = 180 # Forecast for ~6 months
train_end_date = df.index.max() - timedelta(days=forecast_period_days)

# Split data chronologically
train_data = df.loc[df.index <= train_end_date, 'Value']
test_data = df.loc[df.index > train_end_date, 'Value']

print(f"Training data ends on: {train_data.index.max().date()}")
print(f"Testing data starts on: {test_data.index.min().date()}")


# --- Model Training (Exponential Smoothing) ---
# Choose model parameters: trend ('add' or 'mul'), seasonal ('add' or 'mul'), seasonal_periods
# Our simulated data has additive trend and additive weekly seasonality (period=7)
# Using SimpleExpSmoothing would only handle level, Holt's handles trend, Holt-Winters handles trend and seasonality
# ExponentialSmoothing is the flexible model
model = ExponentialSmoothing(train_data,
                             trend='add',           # Additive trend
                             seasonal='add',        # Additive seasonality
                             seasonal_periods=7)    # Weekly seasonality for daily data

# Fit the model
# Use optimize_errors='add' for additive model fitting optimization
fitted_model = model.fit(optimized=True, remove_bias=False) # Optimized=True finds best parameters


# --- Forecasting using .predict() ---
# Generate forecasts for the test period using the .predict() method on the fitted model
# Pass the start and end *dates* (or indices) for the forecast period
forecast_start_date = test_data.index.min()
forecast_end_date = test_data.index.max()

# The .predict() method when used on the *fitted model* (`fitted_model`) with specified dates
# generates the out-of-sample forecast.
forecast_values = fitted_model.predict(start=forecast_start_date, end=forecast_end_date)


# --- Model Evaluation ---
# Evaluate forecasts against actual values in the test set
# Align test data and forecast data index in case of slight date mismatches (though asfreq helps)
actual_test_values = test_data.reindex(forecast_values.index).dropna()
predicted_forecast_values = forecast_values.reindex(actual_test_values.index)


if len(actual_test_values) > 0:
    mae = mean_absolute_error(actual_test_values, predicted_forecast_values)
    mse = mean_squared_error(actual_test_values, predicted_forecast_values)
    rmse = np.sqrt(mse) # Root Mean Squared Error

    evaluation_metrics = pd.DataFrame({
        'Metric': ['Mean Absolute Error (MAE)', 'Mean Squared Error (MSE)', 'Root Mean Squared Error (RMSE)'],
        'Value': [mae, mse, rmse]
    })
else:
    evaluation_metrics = pd.DataFrame({'Result': ["Not enough data in the test set to evaluate forecasts."]})


# --- Visualization ---

# Apply custom style guidelines
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.edgecolor'] = '#1a1a24'
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['axes.grid'] = False # Turn off default grid
sns.set_theme(style="whitegrid") # Use a seaborn theme base, then apply customs


# Plot training data, actual test data, and forecasts
fig1, ax1 = plt.subplots(figsize=(14, 7))

# Plot training data
ax1.plot(train_data.index, train_data, label='Training Data', color='#1a1a24', linewidth=1) # Off-black

# Plot actual test data
ax1.plot(test_data.index, test_data, label='Actual Test Data', color='#188ce5', linewidth=1.5) # Blue

# Plot forecasted values
ax1.plot(forecast_values.index, forecast_values, label='Forecast', color='#ff6d00', linewidth=1.5, linestyle='--') # Orange dashed

ax1.set_title('Time Series Forecasting (Exponential Smoothing)', fontsize=14, color='#1a1a24')
ax1.set_xlabel('Date', fontsize=12, color='#1a1a24')
ax1.set_ylabel('Value', fontsize=12, color='#1a1a24')
ax1.legend()
sns.despine(ax=ax1, top=True, right=True)
ax1.grid(False)
fig1.autofmt_xdate() # Auto-format date labels


plt.tight_layout()


# Output results
# Return a dictionary containing evaluation metrics and the plot
output = {
    'Forecast Evaluation Metrics': evaluation_metrics,
    'Forecast Values Head': forecast_values.head(), # Show head of forecast series
    'Actual Test Values Head': actual_test_values.head(), # Show head of actual test series
    'Time_Series_Forecast_Plot': fig1
}

output # Output the dictionary

**Explanation:**

*   We load the dummy time series data. **Remember to replace `"ForecastData"`**.
*   We convert the 'Date' column to datetime and set it as the DataFrame index.
*   We sort the index to ensure the time series is in chronological order.
*   **Explicitly set Frequency:** `df = df.asfreq('D')` is used to explicitly set the index frequency to daily ('D'). If the original data missed any dates, `asfreq` inserts them with `NaN` values, making the index regular as required by many time series models.
*   **Handle Missing Values:** `df['Value'].interpolate(method='time')` fills `NaN`s using time-weighted linear interpolation, suitable for time series data. A fallback `fillna` is included in case interpolation fails (e.g., at the very start).
*   **Data Splitting:** The data is split into a training set (up to a defined date) and a testing/forecast set (the period after that date). This simulates training on historical data and forecasting/evaluating on unseen future data.
*   **Model Training:** `statsmodels.tsa.holtwinters.ExponentialSmoothing` is initialized. We specify `trend='add'` and `seasonal='add'` based on our simulated data's properties, and `seasonal_periods=7` for daily data with weekly seasonality. `.fit()` trains the model on the `train_data`.
*   **Forecasting with `.predict()`:** **As requested**, we use the `.predict()` method of the *fitted model object* (`fitted_model`) and provide the `start` and `end` dates corresponding to the `test_data` index. This correctly generates out-of-sample forecasts for the specified period.
*   **Model Evaluation:** We calculate `Mean Absolute Error (MAE)` and `Root Mean Squared Error (RMSE)` by comparing the `forecast_values` to the `actual_test_values`. These metrics quantify the average forecast error. We use `reindex` and `dropna` to ensure perfect alignment between the forecast and actual values for evaluation.
*   **Visualization:** A plot shows the original training data, the actual values in the test period, and the predicted forecast values. This visually assesses how well the forecast aligns with the actual future data.
*   **Custom Style:** Applied the specified style guidelines (font, colors - off-black for training data, blue for actual test data, orange dashed for forecast, off-black for text/axes, axes, spines, grid, formatted x-axis for dates).
*   We return a dictionary containing DataFrames for the evaluation metrics and heads of the forecast/actual test Series, and the forecast plot figure.

**Viewing the Output:**

*   Click the Python cell, then click the Python icon/button next to the formula bar.
*   Select "Excel Value" (**Ctrl+Shift+Alt+M**) for the DataFrames/Series ('Forecast Evaluation Metrics', 'Forecast Values Head', 'Actual Test Values Head') to spill them into your sheet.
*   For the plot figure object ('Time_Series_Forecast_Plot'), select "Picture in Cell" > "Create Reference" to see the plot.

This demonstrates how to build and evaluate a basic time series forecasting model using `statsmodels` and the `.predict()` method.

**Further Analysis:**

Here are some advanced analyses you could perform on this time series dataset:

1. **Advanced Forecasting Models:**
   - Implement SARIMA (Seasonal ARIMA) models for complex seasonality patterns
   - Use Prophet for automatic handling of holidays and multiple seasonal patterns
   - Apply LSTM or other deep learning models for non-linear time series patterns

2. **Multi-variate Time Series Analysis:**
   - Include external regressors (e.g., weather data, economic indicators)
   - Implement VAR (Vector Autoregression) models
   - Study cross-correlations between multiple time series

3. **Anomaly Detection:**
   - Implement statistical process control charts
   - Use isolation forests or other ML-based anomaly detection
   - Develop real-time monitoring systems

4. **Decomposition Analysis:**
   - Compare different decomposition methods (additive vs. multiplicative)
   - Study trend-cycle separation techniques
   - Analyze multiple seasonal patterns at different frequencies

5. **Forecasting Performance Analysis:**
   - Implement rolling-window cross-validation
   - Compare different error metrics (MAPE, RMSE, MAE)
   - Study forecast uncertainty using prediction intervals

The next topic in the series is [Visualization - Basic Plots (Line, Bar, Scatter)](./06-Visualization_01-Basic%20Plots%20(Line,Bar,Scatter).md), which explores fundamental data visualization techniques using Python plotting libraries.

This completes the Time Series Forecasting section. The next category covers **Visualization**, where you'll learn to create various types of visual representations of your data. For additional practice, try implementing different time series models or refining the forecasting approach with your own data.